In [1]:
import sys
sys.path.append('../input/bird-tools')

In [2]:
import noisereduce as nr

In [3]:
PATH_DATA = "../input/birdclef-2022"
class config:
    seed=2022
    num_fold = 5
    sample_rate= 32_000
    sampleNum = 32_000*5
    n_fft=1024
    win_length = 1024
    hop_length=512
    n_mels=64
    duration=5
    num_classes = 152
    train_batch_size = 16
    valid_batch_size = 16
    epochs = 50
    learning_rate = 1e-3

# Load Model

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# CNN
# class BirdClefModel(nn.Module):
#     def __init__(self):
#         super(BirdClefModel, self).__init__()
#         self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=1)
#         self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1)
#         self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
#         self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)

#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.fc1 = nn.Linear(64*32*19, 1024*10)
#         self.fc2 = nn.Linear(1024*10, 1024)
#         self.fc3 = nn.Linear(1024, config.num_classes)
#         self.softmax = nn.Softmax(dim=1)
#         self.dropout = nn.Dropout()

#     def forward(self, x):
#         x = F.relu(self.conv1(x))
#         # 513*313
#         x = self.pool(x)
#         x = F.relu(self.conv2(x))
#         # 256*156
#         x = self.pool(x)
#         x = F.relu(self.conv3(x))
#         # 128*78
#         x = self.pool(x)
#         # 64*39
#         x = F.relu(self.conv4(x))
#         x = self.pool(x)
#         # 32*19
        
#         x = x.view(x.size(0), -1)
# #         x = self.dropout(x)
#         x = F.relu(self.fc1(x))
# #         x = self.dropout(x)
#         x = F.relu(self.fc2(x))
# #         x = self.dropout(x)
#         x = self.fc3(x)
# #         x = self.softmax(x)
#         return x

class BirdClefModel(nn.Module):
    def __init__(self):
        super(BirdClefModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128*16*9, 1024*4)
#         self.fc2 = nn.Linear(1024*8, 1024)
        self.fc3 = nn.Linear(1024*4, config.num_classes)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        # 513*313
        x = self.pool(x)
        # 256*156        
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        # 128*78
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        # 64*39
        x = F.relu(self.conv4(x))
        x = self.pool(x)
        # 32*19
        x = F.relu(self.conv5(x))
        x = self.pool(x)
        # 16*9
        
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
#         x = F.relu(self.fc2(x))
#         x = self.dropout(x)
        x = self.fc3(x)
    
        return x

In [5]:
import os

PATH_Model = "../input/model-02f1"

model_path = os.path.join(PATH_Model, "model.pt")
model = BirdClefModel()
model.load_state_dict(torch.load(model_path,map_location='cpu'))
model.eval()
print(model)

BirdClefModel(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=18432, out_features=4096, bias=True)
  (fc3): Linear(in_features=4096, out_features=152, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)


# Deal Audio Data

In [6]:
7//2

3

In [7]:
import noisereduce as nr
import torchaudio
# STFT
device = "cpu"
n_fft = 1024
win_length = 1024
hop_length = 512
transform = torchaudio.transforms.Spectrogram(
    n_fft = n_fft,           # freqGroup = n_fft//2 + 1
    win_length = win_length, # freq gap for each group
    hop_length = hop_length, # length = samples / hop_length
    center = True,
    pad_mode = 'reflect',
    power=2.0
).to(device)

@torch.no_grad()
def create_spectrogram(fname,reduce_noise = False,channel = 0):
    waveform, sample_rate = torchaudio.load(fname)
       
    # Change singal to mono
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, axis=0, keepdim=True)
        
    # Reduce noise
    if reduce_noise:
        waveform = torch.tensor(nr.reduce_noise(
            y=waveform,
            sr=sample_rate,
            use_tqdm=True,
            n_jobs=3,
        ))
    step = int(5 * sample_rate)
    lenSamples = waveform.size()[-1]
    if (lenSamples%step) > (step*0.6):
        waveform = torch.nn.functional.pad(waveform, (0,step-lenSamples%step), mode='constant', value=0.0)
    

    frames = []
    for i in range(waveform.size()[-1]//step):
        begin = i * step
#         print(output_path)
        frame = waveform[:,begin:begin + step]
        frames.append(frame)
    return transform(torch.stack(frames)) 
        

# path_audio = '../input/birdclef-2022/test_soundscapes/soundscape_453028782.ogg'
# print(path_audio)
# spec = create_spectrogram(path_audio, reduce_noise=True)
# print(spec.shape)

# Test

In [8]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
df = pd.read_csv('../input/birdclef-2022/train_metadata.csv')

## Encoder

In [9]:
encoderList = np.load('../input/model-02f1/encoder_list.npy',allow_pickle=True)
encoder = LabelEncoder()
encoder.fit(encoderList)
# encoder.classes_

LabelEncoder()

## Simple Train Test

In [10]:
# import IPython.display as ipd

# index = 9
# testData = df.loc[index]
# path_audio = os.path.join(PATH_DATA,'train_audio',testData.filename)
# print(testData.primary_label,testData.secondary_labels,testData.type,testData.rating)
# display(ipd.Audio(path_audio))



# spec = create_spectrogram(path_audio, reduce_noise=True)

# outputs = model(spec)
# _, preds = torch.max(outputs, 1)
# print(encoder.inverse_transform(preds))

## Test Example

In [11]:
# path_audio = '../input/birdclef-2022/test_soundscapes/soundscape_453028782.ogg'

# spec = create_spectrogram(path_audio, reduce_noise=True)

# outputs = model(spec)

# _, preds = torch.max(outputs, 1)

# print(encoder.inverse_transform(preds))
# outputs.shape

## Test Submission

In [12]:
import json

with open(os.path.join(PATH_DATA, "scored_birds.json")) as fp:
    scored_birds = json.load(fp)

print(scored_birds)
print(encoder.transform(scored_birds))

['akiapo', 'aniani', 'apapan', 'barpet', 'crehon', 'elepai', 'ercfra', 'hawama', 'hawcre', 'hawgoo', 'hawhaw', 'hawpet1', 'houfin', 'iiwi', 'jabwar', 'maupar', 'omao', 'puaioh', 'skylar', 'warwhe1', 'yefcan']
[  3   6   7   9  44  46  47  60  62  63  64  65  67  70  72  90 101 111
 131 141 150]


In [13]:
# import glob
# path_audio = glob.glob("/kaggle/input/birdclef-2022/test_soundscapes/*.ogg")[0]
# print(path_audio)
# spec = create_spectrogram(path_audio, reduce_noise=True)
# segSpec = cut_Spectrum(spec,segLen)
# outputs = model(segSpec)


In [14]:
import glob
import math

TestPathAudio = glob.glob("/kaggle/input/birdclef-2022/test_soundscapes/*.ogg")
threshold = 1/config.num_classes
outputBirds = list(zip(scored_birds,encoder.transform(scored_birds)))

submission = []
for path_audio in TestPathAudio:
    filename = os.path.basename(path_audio).replace('.ogg','')

    spec = create_spectrogram(path_audio, reduce_noise=True)
    outputs = model(spec)
    outputs = torch.nn.Softmax(dim=1)(outputs)
    
    for i in range(len(outputs)):
        for bird,pos in outputBirds:
            submission.append({
                "row_id": filename + '_'+bird +'_' + str((i+1)*5),
                "target": outputs[i,pos].item() > threshold,
            })

In [15]:
df_submission = pd.DataFrame(submission).set_index("row_id")
df_submission.to_csv("submission.csv")


In [16]:
df_submission

,target
row_id,
soundscape_453028782_akiapo_5,False
soundscape_453028782_aniani_5,False
soundscape_453028782_apapan_5,False
soundscape_453028782_barpet_5,False
soundscape_453028782_crehon_5,False
...,...
soundscape_453028782_omao_60,False
soundscape_453028782_puaioh_60,False
soundscape_453028782_skylar_60,False
